## Preparing data for multivariate regression analysis.  Cleaning NYC crime data.  Some rows need to be deleted, columns dropped and some data needs to be converted to categorical instead of continuous.

In [6]:
# importing pnadas modules
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [7]:
# importing raw data from excel into a pandas dataframe
df_raw = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

# display raw file
df_raw.head(n=10)

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0
5,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0
6,Akron Village,2846,3,0,NaN,0,0,3,16,1,15,0,0
7,Albany,97956,791,8,NaN,30,227,526,4090,705,3243,142,NaN
8,Albion Village,6388,23,0,NaN,3,4,16,223,53,165,5,NaN
9,Alfred Village,4089,5,0,NaN,0,3,2,46,10,36,0,NaN


As you can see in the table above, there are unnecessary rows and 'NaN' values.  We need to clean this dataframe.

In [8]:
# show last 5 lines of raw file; last 3 lines need to be removed
df_raw.tail()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
350,Yonkers,199134,1036,6,NaN,25,390,615,2368,470,1662,236,10
351,Yorktown Town,36643,15,0,NaN,0,2,13,334,45,287,2,NaN
352,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,2 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,3 The FBI does not publish arson data unless i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The last few rows of this raw data are just for explanatory reasons and does not contain any actual data.

In [9]:
# setting the proper heading from the 4th row of raw data
headers = df_raw.iloc[3]

# exclude rows 1 through 4 (or rows 0, 1, 2, 3)
df_ny = pd.DataFrame(df_raw.values[4:], columns=headers)

# renaming columns
df_ny.columns = ['city', 'pop', 'vc', 'murder', 'rape1', 'rape2', 'robbery', 'aa', 'pc', 'burglary', 'lt', 'mvt', 'arson3']

# dropping unnecessary columns
df_ny2 = df_ny.drop(['vc', 'rape1', 'rape2', 'aa', 'pc', 'burglary', 'lt', 'mvt', 'arson3'], axis=1)

# removing last 3 rows of instructional data
df_ny3 = df_ny2.drop([348, 349, 350], axis=0)


In [10]:
df_ny3.head()


,city,pop,murder,robbery
0,Adams Village,1861,0,0
1,Addison Town and Village,2577,0,0
2,Akron Village,2846,0,0
3,Albany,97956,8,227
4,Albion Village,6388,0,4


Now, you can see that we have dropped some unnecessary columns as we are only concerned with the city, population, murder and robbery.  These columns have been renamed from the raw data for ease of use and convenience.  We also dropped the last 3 explanatory rows from the dataframe.

In [11]:
#converting string values to float and creating 'pop2' column which is pop squared

df_ny3["pop"] = [np.float(x) for x in df_ny3["pop"]]
df_ny3["pop2"] = [np.float(x*x) for x in df_ny3["pop"]]
df_ny3["murder"] = [np.float(x) for x in df_ny3["murder"]]
df_ny3["robbery"] = [np.float(x) for x in df_ny3["robbery"]]


In [12]:
df_ny3.dtypes

city        object
pop        float64
murder     float64
robbery    float64
pop2       float64
dtype: object

In [13]:
# If murders or robberies in greater than 0, convert to 1 else leave at 0
df_ny3['murder'] = df_ny3['murder'].apply(lambda x : 1 if x>0 else 0)
df_ny3['robbery'] = df_ny3['robbery'].apply(lambda x : 1 if x>0 else 0)
df_clean = df_ny3

In [14]:
df_clean.head()

,city,pop,murder,robbery,pop2
0,Adams Village,1861.000,0,0,3463321.000
1,Addison Town and Village,2577.000,0,0,6640929.000
2,Akron Village,2846.000,0,0,8099716.000
3,Albany,97956.000,1,1,9595377936.000
4,Albion Village,6388.000,0,1,40806544.000


The final step in preparing this data was to convert any instance of murder or robbery to a value of 1.  In other words, if the number of murders was 8 for murder in Albany, we converted this to just a value of 1 - to use for a categorical variable later.  This is done to signify that at least 1 murder did occur (same with robberies).  That is all we are interested in at this point.